In [ ]:
import itertools
import json
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pygmo
from pymoo.indicators.igd_plus import IGDPlus
from datetime import datetime
import time
import seaborn as sns
import copy
from matplotlib.lines import Line2D

import scenarios as scenariodef
from multiprocess import Pool
# import configurators

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger().setLevel(logging.WARNING)

resultdir = Path("results")
figurepath = Path("figures")
figurepath.mkdir(parents=True, exist_ok=True)

## Style Definitions

In [ ]:
sns.set_style("whitegrid")

configurators = ["RandomSearch", "RandomSearchSI", "RandomSearch25", "default",  "SMAC", "MO-ParamILS",  "ParEGO", "MO-SMAC", "MO-SMAC-PHVI"]

# color_palette = [(230, 159, 0), (86, 180, 233), (0, 158, 115), (240, 228, 66), (0, 114, 178), (213, 94, 0), (204, 121, 167)]  # Wong color palette (works for color-blinded people)
color_palette = [(51, 34, 136), (17, 119, 51), (68, 170, 156), (136, 204, 238), (221, 204, 119), (204, 102, 119), (170, 68, 153), (136, 34, 85),][::-1]  # Tol colorblind palette (https://davidmathlogic.com/colorblind/#%23332288-%23117733-%2344AA99-%2388CCEE-%23DDCC77-%23CC6677-%23AA4499-%23882255)
color_palette = [tuple([float(f"{channel / 255:.3f}") for channel in color]) for color in color_palette]
color_palette += color_palette
# color_palette = sns.color_palette(palette="colorblind")
markers = ["s", "o", "X", "P", "^", "v", ">", "<", "*"][::-1]



def get_style(conf, key):
    return style_guide[conf][key]

# display(style_guide)

config_names = {
    "MO-SMAC-PHVI": "MO-SMAC-PHVI",
    "ParEGO":  "MO-SMAC-PE",
    "MO-SMAC": "MO-SMAC-EHVI",
    "MO-ParamILS": "MO-ParamILS",
    "SMAC": "SMAC",
    "RandomSearchSI": "Random Search + MO-Intensify",
    "RandomSearch": "Random Search",
    "RandomSearch25": "Random Search",
    "default": "Default",
}

style_guide = {}
for i, conf in enumerate(config_names.values()):
    style_guide[conf] = {"color": color_palette[i%len(color_palette)], "marker": markers[i%len(markers)]}

print(style_guide)

# import matplotlib
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

# #matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     #"pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     # 'text.usetex': True,
#     # 'pgf.rcfonts': False,
# })

scenario_replacements = {
    "EA_OO_MMMOOP": "EMOA-(hv, sp)",
    "MIP_CPLEX_REGIONS200_cutoff": "MIP-(gap, cutoff)",
    "MIP_CPLEX_REGIONS200_runtime": "MIP-(gap, runtime)",
    "MIP_CPLEX_REGIONS200_CONT_cutoff": "MIP-(gap, cutoff) (c)",
    "MIP_CPLEX_REGIONS200_CONT_runtime": "MIP-(gap, runtime) (c)",
    "SAT_CMS_QUEENS_runtime_memory": "SAT-(runtime, memory)",
    "SAT_CMS_QUEENS_runtime_solved": "SAT-(runtime, solved)",
    "ML_RF_STUDENTS_precision_recall": "ML-(precision, recall)",
    "ML_RF_STUDENTS_precision_recall_size": "ML-(precision, recall, size)",
    "ML_RF_STUDENTS_accuracy_size": "ML-(accuracy, size)",
}

configurator_replacements = config_names
# {
#     "RandomSearch": "Random Search",
#     "RandomSearchSI": "Random Search + MO-Intensify",
#     "default": "Default",  
#     # "SMAC": ,
#     # "MO-ParamILS",:  
#     "ParEGO": "MO-SMAC-PE",
#     "MO-SMAC-ParEGO": "MO-SMAC-PE",
#     "MO-SMAC": "MO-SMAC-EHVI",
#     # "MO-SMAC-PHVI": ,
# }

palette = {conf: col for conf, col in zip(config_names.values(), color_palette)}
markers = {conf: col for conf, col in zip(config_names.values(), markers)}
order = configurators  #  [conf for conf in list(config_names.keys()) if conf in list(qdf["configurator"].unique())]


def rename_metadata(df: pd.DataFrame) -> pd.DataFrame:
    for k, v in scenario_replacements.items():
        df.loc[df["scenario"] == k, "scenario"] = v
    for k, v in configurator_replacements.items():
        df.loc[df["configurator"] == k, "configurator"] = v
    return df

print(f"{config_names=}")
print(f"{palette=}")
print(f"{markers=}")



### Legend

In [ ]:
ncols = len(configurators)

fig = plt.figure(figsize=(ncols, 1), dpi=300)
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
handles = []
handles_small = []
legend_names = []

for basename, conf in config_names.items():
    if basename in ["MO-SMAC", "RandomSearchSI", "RandomSearch"]:
        continue
    style = style_guide[conf]
    print(conf, style)
    handles.append(Line2D([0], [0], linestyle="", markersize=14, **style))
    handles_small.append(Line2D([0], [0], linestyle="", markersize=10, **style))
    legend_names.append(configurator_replacements.get(conf, conf))

fig.legend(handles, legend_names, loc="upper center", fontsize=18, frameon=False, columnspacing=1.15, ncol=ncols)  # ncols=ncols
fig.tight_layout()
fig.savefig(figurepath / "legend.pdf", bbox_inches="tight", dpi=300, pad_inches=0)
plt.show()

textwidth = 404 * 2.4# pt
pt_per_in = 0.0138888889 
textwidth_in = textwidth * pt_per_in
n_figs_per_row = 4.25
figwidth = textwidth_in / n_figs_per_row
figsize = (figwidth, figwidth * 0.8)
fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
fig.legend(handles_small, legend_names, loc="center", fontsize=11, frameon=False, columnspacing=1.15, ncol=1)  # ncols=ncols
# fig.tight_layout()
fig.savefig(figurepath / "legend2.pdf", bbox_inches="tight", dpi=300, pad_inches=0)
plt.show()

# Performance results
Configuration procedure:
- Run a configurator $n$ times on a scenario with a different random seed. (We used $n=10$)
- Validate the final incumbent (single or multiple configurations) of each configuration run on a common validation set (subset of the train set).
- Combine all the incumbents into one archive and throw out all the dominated configurations. This configuration in the archive are non-dominated.
- Validate configurations in the filtered archive on the training set.

HV computation:
- Determine reference point by choosing the largest obtained mean value for an objective by all validation and training runs of all configurations found by all configurators.
- Compute the HV.

## Dataloading

In [ ]:
scenario_files = [d for d in resultdir.joinpath("collect").iterdir()]
scenarios = []
for i, s in enumerate(scenario_files):
    print(f"###### {s}")
    dfpath = f"{resultdir}/collect/dataframes/{Path(s).name}.csv"
    metapath = f"{resultdir}/collect/metadata/{Path(s).name}.json"

    loaded_from_csv: bool = False
    try:
        with open(s, "rb") as fh:
            scenario = pickle.load(fh)
    except:
        # try json and csv
        print("Try csv")
        if not (Path(dfpath).exists() and Path(metapath).exists()):
            print(f"Couldn't load {s}")
            continue

        df = pd.read_csv(dfpath)
        loaded_from_csv = True
        with open(metapath, "r") as f:
            scenario = json.load(f)

        scenario["run_result"] = df
        pass
    
    scenario_name = str(scenario['experiment']["scenarios"])
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]

    print(f"> {scenario['experiment']['scenarios']}")

    if scenario["experiment"]["scenarios"] in ["MIP_CPLEX_REGIONS200_CONT_cutoff",
                                               "MIP_CPLEX_REGIONS200_CONT_runtime",
                                               "SAT_CMS_QUEENS_runtime_memory",
                                               "ML_RF_STUDENTS_precision_recall",
                                               "ML_RF_STUDENTS_precision_recall_size",
                                               "ML_RF_STUDENTS_accuracy_size",
                                               "EA_OO_MMMOOP"
                                               ]:
        df = scenario["run_result"]
        df = df[~df["configurators"].isin(["MO-SMAC", "RandomSearchSI", "RandomSearch"])]

        df["seeds"] = df["seeds"].astype(int)
        print("seeds", list(df["seeds"].unique()))

        df = df[df["seeds"] <  20]
        df.loc[df["status"] == "CRASHED", objectives] = df[df["status"] != "CRASHED"][objectives].max()
        scenario["run_result"] = df
        scenarios.append(scenario)

        #export
        for p in [dfpath, metapath]:
            Path(p).parent.mkdir(parents=True, exist_ok=True )
        df = scenario["run_result"]
        if not loaded_from_csv:
            df.to_csv(dfpath, index=False)
        meta_data = copy.copy(scenario)
        del meta_data["run_result"]
        with open(metapath, "w") as f:
            json.dump(meta_data, f)
    
    else:
        print(">> Skipping..")

print(f"Got {len(scenarios)} scenarios")

for scenario in scenarios:
    for k, v in scenario_replacements.items():
        scenario["run_result"].loc[scenario["run_result"]["scenarios"] == k, "scenarios"] = v
    for k, v in configurator_replacements.items():
        scenario["run_result"].loc[scenario["run_result"]["configurators"] == k, "configurators"] = v
    

scenarios = sorted(scenarios, key=lambda s: s["experiment"]["scenarios"])  # Sort by name

[s["experiment"]["scenarios"] for i, s in enumerate(scenarios)]

In [ ]:
[s["experiment"]["scenarios"] for i, s in enumerate(scenarios)]

In [ ]:
### Sanity checking
for scenario in scenarios:
    scenario_name = str(scenario['experiment']["scenarios"])
    # if scenario_name in ["SAT_CMS_QUEENS_runtime_memory"]:
    #     continue
    print(f"{'scenario':15}: {scenario_name}")
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]
    print(f"{'objectives':15}: {objectives}")

    df = scenario["run_result"]
    print(f"{'configurators':15}: {df['configurators'].unique()}")

    print(df.groupby("configurators")["seeds"].unique())
    print(df.groupby(["configurators","status"]).size())
    # print(df.groupby(["configurators","action"])["instance"].nunique())
    # print(df.groupby(["configurators","seeds","configuration"])["instance"].size())

#     print("-"*64)

In [ ]:
# keys = ["configurators", "seeds", "configuration"]
# perf = df.groupby(keys)[["PAR10", "memory"]].mean()
#
# best_run  = perf.sort_values("PAR10").index[0]
#
# df[list(map(lambda x: all(x), zip(*[df[k] == v for k, v in zip(keys, best_run)])))]
df

In [ ]:
# Timeouts
param = []
keys = ["action", "configurators", "seeds", "configuration"]
tdf = df.sort_values(keys).groupby(keys)["status"].value_counts().unstack("status")
tdf["Percentage TO"] = tdf["TIMEOUT"] / (tdf["TIMEOUT"] + tdf["SUCCESS"])

tdf.reset_index().groupby(["action", "configurators"])["Percentage TO"].min()

In [ ]:
df = scenarios[0]["run_result"]

df.groupby(["configurators","seeds","configuration"])["PAR10"].min().unstack("configurators")

## Selection of final front and performance computation

In [ ]:
scenario = scenarios[-1]
scenario_name = str(scenario['experiment']["scenarios"])
print(scenario_name)
scen = getattr(scenariodef, scenario_name)()
objectives = [o["name"] for o in scen.objectives]

df = scenario['run_result']
performances = df.groupby(["action","configurators", "seeds", "configuration"])[objectives].mean().unstack("action").swaplevel(0, 1, 1)
performances.describe()

In [ ]:
performances = df.groupby(["action", "seeds", "configurators", "configuration"])[objectives].mean().reset_index()
stats = performances.groupby("action")[objectives].describe().stack()
stats

In [ ]:
sns.set(font_scale=1.)

hvs = []
igdp_dict = {}
partitions = ["validate", "test"]
for scenario, partition in itertools.product(scenarios, partitions):
    scenario_name = str(scenario['experiment']["scenarios"])
    # if scenario_name not in ["MIP_CPLEX_REGIONS200_cutoff_50", "MIP_CPLEX_REGIONS200_runtime_50", "SAT_CMS_QUEENS_runtime_memory_50"]:
    #     continue
    print(scenario_name)
    print(scenariodef)
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]
    print(objectives)

    df = scenario["run_result"]
    print(len(df[df["configurators"] == "Default"]))
    df = df[~((df["configurators"] == "Default") & (df["seeds"] != 0))]
    print(len(df[df["configurators"] == "Default"]))
    if "ML_RF_STUDENTS_accuracy" in scenario_name:
        # Remove F1 from this scenario
        df = df[~((df["configurators"] == "SMAC") & (df["configuration"] == 2))]
    performances = df.groupby(["action","configurators", "seeds", "configuration"])[objectives].mean().unstack("action").swaplevel(0, 1, 1)

    minimum_val = performances["validate"].min()
    maximum_val = performances["validate"].max()

    minimum = performances[partition].min()
    maximum = performances[partition].max()

    print(f"{maximum=}")
    print(f"{minimum=}")
    
    #Export for trajectories
    stats = df.groupby(["action", "seeds", "configurators", "configuration"])[objectives].mean().reset_index()
    stats = stats.groupby("action")[objectives].describe().stack()
    stats_dest = Path("intermediates/norm_stats/")
    stats_dest.mkdir(exist_ok=True, parents=True)
    stats.to_csv(stats_dest / f"{scenario_name}.csv")

    points = performances[partition].to_numpy()
    #Normalize objectives
    for i, o in enumerate(objectives):
        points[:, i] = 1 + ((points[:, i] - minimum[o]) / (maximum[o] - minimum[o]))
    reference_set_ids = pygmo.fast_non_dominated_sorting(points)[0][0]
    reference_set = points[reference_set_ids, :]

    print(f"{reference_set.shape=}")
    
    fig, ax = plt.subplots(figsize=figsize)
    raw_reference_set = copy.copy(reference_set)
    for i, o in enumerate(objectives):
        raw_reference_set[:, i] = minimum[o]+((raw_reference_set[:,i]-1)*(maximum[o]-minimum[o]))
    ax.scatter(*raw_reference_set.T)
    ax.set_title(f"{scenario_replacements[str(scen)]}")
    ax.set_xlabel(objectives[0])
    ax.set_ylabel(objectives[1])
    if "ML" in str(scen):
        if objectives[o1] in ["accuracy", "recall", "precision"]:
            ax.set_xlabel(f"1 - {objectives[0]}")
        if objectives[o2] in ["accuracy", "recall", "precision"]:
            ax.set_ylabel(f"1 - {objectives[1]}")
    if "MIP" in str(scen) and "runtime" in str(scen):
        ax.set_ylim((0,10))
    if objectives[1] == "size" and "ML_RF" in str(scen):
        ax.set_yscale("log")
    fig.savefig(figurepath / f"{partition}_fronts/refset_{scen}.pdf", dpi=150, bbox_inches='tight', pad_inches=0)
    fig.show()
    # fig.clf()

    igdplus = IGDPlus(reference_set)
    igdp_dict[(scenario_name, partition)] = igdplus

    figures = []
    for _ in itertools.combinations(range(len(objectives)), r=2):
        figures.append(plt.subplots(figsize=figsize))

    # Sort configurators
    performances["order"] = performances.apply(lambda r: list(config_names.values()).index(r.name[0]) , axis=1)
    performances = performances.sort_values("order")

    for (_, configurator), pdf in performances.groupby(["order", "configurators"]):
        print(configurator)
        #Compute ND set from validation performance
        points = pdf["validate"].to_numpy()
        if len(points) > 1:
            ndf, dl, dc, ndr = pygmo.fast_non_dominated_sorting(points)
            front = ndf[0]
        else:
            front = [0]

        #Get test performance
        points = pdf[partition].to_numpy()
        if np.any(np.isnan(points)):
            print(f"MISSING POINTS, SKIPPING {configurator}")
            continue

        # Plot performance
        for figid, objids in enumerate(itertools.combinations(range(len(objectives)), r=2)):
            ndpoints = points[:, objids]
            ndpoints = ndpoints[front, :]
            ndpoints = ndpoints[np.argsort(ndpoints[:, 0])]

            highlightpoints = ndpoints
            dominatedpoints = []
            if len(ndpoints) > 1:
                ndpi = np.zeros(len(ndpoints), dtype=bool)
                for i in pygmo.fast_non_dominated_sorting(ndpoints)[0][0]:
                    ndpi[i] = True
                highlightpoints = ndpoints[ndpi, :]
                dominatedpoints = ndpoints[~ndpi, :]

            highlightpoints = highlightpoints[np.argsort(highlightpoints[:, 0])]
            steps = [highlightpoints[0,:]]
            for i in range(1, len(highlightpoints)):
                steps.append((highlightpoints[i,0], highlightpoints[i-1,1]))
                steps.append(highlightpoints[i])
            figures[figid][1].scatter(
                *highlightpoints.T, 
                label=configurator, 
                marker=get_style(configurator, "marker"), 
                color=get_style(configurator, "color")
            )
            figures[figid][1].plot(*zip(*steps), color=get_style(configurator, "color"), alpha=0.25, linewidth=2)
            if len(dominatedpoints) > 0:
                figures[figid][1].scatter(*dominatedpoints.T, marker=get_style(configurator, "marker"), color=get_style(configurator, "color"), alpha=0.25)


        #Compute HV and IGD+
        #Normalize
        ndpoints = points[front, :]
        for i, o in enumerate(objectives):
            ndpoints[:, i] = 1 + ((ndpoints[:, i] - minimum[o]) / (maximum[o] - minimum[o]))

        hv = pygmo.hypervolume(ndpoints).compute([2.0+1e-1]*len(objectives))
        igdp_score = igdplus(ndpoints)
        hvs.append({"scenario": scenario_name,
                    "configurator":configurator,
                    "partition": partition,
                    "hypervolume": hv,
                    "igdp": igdp_score})



    for figid, (o1, o2) in enumerate(itertools.combinations(range(len(objectives)), r=2)):
        fig, ax = figures[figid]
        # ax.set_title(scen)
        ax.set_xlabel(objectives[o1])
        ax.set_ylabel(objectives[o2])
        if "ML" in str(scen):
            if objectives[o1] in ["accuracy", "recall", "precision"]:
                ax.set_xlabel(f"1 - {objectives[o1]}")
            if objectives[o2] in ["accuracy", "recall", "precision"]:
                ax.set_ylabel(f"1 - {objectives[o2]}")
        if "MIP" in str(scen) and "runtime" in str(scen):
            ax.set_ylim((0,10))
        if objectives[o2] == "size" and "ML_RF" in str(scen):
            ax.set_yscale("log")

        # ax.set_aspect("equal")
        # ax.set_aspect('equal', adjustable='datalim')
        sns.set_style("whitegrid")

        # ax.legend()
        # ax.set_title(f"{str(scen)[:3].replace('_','')}-({objectives[o1]},{objectives[o2]})")
        ax.set_title(f"{scenario_replacements[str(scen)]}")
        # ax.set_title(f"{str(scen)[:3].replace('_','').replace(' (c)','')}-({', '.join(objectives)})")
        fig.tight_layout()
        Path(figurepath / f"{partition}_fronts/").mkdir(exist_ok=True, parents=True)
        fig.savefig(figurepath / f"{partition}_fronts/{scen}_{o1}_{o2}.pdf", dpi=150, bbox_inches='tight', pad_inches=0)
        fig.show()
        fig.clf()
plt.close()
qdf = pd.DataFrame(hvs)

for k, v in scenario_replacements.items():
    qdf.loc[qdf["scenario"] == k, "scenario"] = v

qdf

## Tables

In [ ]:
qdf

In [ ]:
# Generate Tables
def formatt(s):
    return s.replace("_", "\_")

fqdf = qdf[~qdf["configurator"].isin(["MO-SMAC", "Random Search"])]
configorder = ["MO-SMAC-PHVI", "MO-SMAC-PE", "MO-ParamILS", "Random Search (25)", "SMAC", "Default"]
for (performance_measure, maximise, notation), (p, grp) in itertools.product([("hypervolume", True, ".3f"), ("igdp", False, ".2e")], fqdf.groupby("partition")):
    print(p, performance_measure)
    grp = grp.set_index(["scenario", "configurator"])[performance_measure].unstack("configurator")
    grp = grp[configorder]
    display(grp)

    table = "\\begin{tabular}{l" + "r"*grp.shape[1] + "} \\toprule \n"
    table += " & " + " & ".join([f"\\bf{{{formatt(h)}}}" for h in grp.columns]) + "\\\\\n"
    table += "\\midrule \n"
    for scenario, row in grp.iterrows():
        best_val = row.max() if maximise else row.min()
        ranks = -row if maximise else row
        ranks = np.argsort(np.argsort(ranks))
        ranks = 1 + ranks
        table += f"\\bf{{{formatt(str(scenario))}}}" + " & " + " & ".join([f"{v:.2e} ({r})" if v != best_val or np.isnan(v) else f"\\bf{{{v:.2e}}} ({r})" for v, r in zip(row, ranks)]) + "\\\\\n"
    table += "\\bottomrule \n\\end{tabular}\n"

    table_fn = Path(f"tables/{performance_measure}_{p}.tex")
    table_fn.parent.mkdir(exist_ok=True, parents=True)
    with open(table_fn, "w") as fh:
        fh.write(table)


## Performance stability of individual runs

In [ ]:
hvs = []
partitions = ["validate", "test"]
for scenario, partition in itertools.product(scenarios, partitions):
    print()
    scenario_name = str(scenario['experiment']["scenarios"])
    # if scenario_name not in ["MIP_CPLEX_REGIONS200_cutoff_50", "MIP_CPLEX_REGIONS200_runtime_50", "SAT_CMS_QUEENS_runtime_memory_50"]:
    #     continue
    print(scenario_name)
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]
    print(objectives)

    igdplus = igdp_dict[(scenario_name, partition)]

    df = scenario["run_result"]
    #df = df[~((df["configurators"] == "default") & (df["seeds"] != "0"))]
    if "ML_RF_STUDENTS_accuracy" in scenario_name:
        # Remove F1 from this scenario
        df = df[~((df["configurators"] == "SMAC") & (df["configuration"] == 2))]
    performances = df.groupby(["action","configurators", "seeds", "configuration"])[objectives].mean().unstack("action").swaplevel(0, 1, 1)
    minimum = performances.stack("action").min()
    maximum = performances.stack("action").max()

    print("max:", maximum)
    print("min:", minimum)

    print(df.columns)
    print("configurators", df["configurators"].unique())

    for (configurator, seed), pdf in performances.groupby(["configurators", "seeds"]):
        if configurator == "MO-SMAC":
            continue
        #Get test performance
        points = pdf[partition].to_numpy()
        #Compute HV
        #Normalize
        ndpoints = points
        for i, o in enumerate(objectives):
            ndpoints[:, i] = 1 + ((ndpoints[:, i] - minimum[o]) / (maximum[o] - minimum[o]))

        hv = pygmo.hypervolume(ndpoints).compute([2.0+1e-10]*len(objectives))
        igdp_score = igdplus(ndpoints)
        hvs.append({"scenario": scenario_name,
                    "configurator":configurator,
                    "seed": seed,
                    "partition": partition,
                    "hypervolume": hv,
                    "igdp": igdp_score})



qdf = pd.DataFrame(hvs)
for k, v in scenario_replacements.items():
    qdf.loc[qdf["scenario"] == k, "scenario"] = v

In [ ]:
extremes = qdf.groupby(["partition","scenario"])[["hypervolume", "igdp"]].agg(["min", "max"])
# qdf.apply(lambda r: extremes[(r["partition"], r["scenario"])], axis=1)
def normalize(row):
    se = extremes.loc[(row["partition"], row["scenario"])]
    for p in ["hypervolume", "igdp"]:
        row[p] =  (row[p] - se[(p, "min")]) / (se[(p, "max")] - se[(p, "min")])
    return row

nqdf = qdf.apply(normalize, axis=1)

In [ ]:
qdf[(qdf["partition"] == "test") & (qdf["scenario"] == "SAT_CMS_QUEENS_runtime_memory") & (qdf["configurator"] == "SMAC")]

### Statistical test (not used in paper)

In [ ]:
import scipy

In [ ]:
candidates = list(qdf["configurator"].unique())

for quality in ["hypervolume", "igdp"]:
    partition = "test"
    scenario = "MIP_CPLEX_REGIONS200_cutoff"
    table = "\\begin{tabular}{l" + "r"*len(candidates) + "} \\toprule \n"
    for scenario in qdf["scenario"].unique():
        print(scenario)
        perf = qdf[(qdf["partition"] == partition) & (qdf["scenario"] == scenario)].set_index(["configurator", "seed"])[["hypervolume","igdp"]].unstack("configurator")

        p_values = []
        for c1, c2 in itertools.product(list(qdf["configurator"].unique()), repeat=2):
            test = scipy.stats.mannwhitneyu(perf[(quality, c1)], perf[(quality, c2)], alternative="two-sided", method="exact")
            p_values.append({"config1": c1, "config2": c2, "p_value": test.pvalue})

        p_values = pd.DataFrame(p_values).set_index(["config1", "config2"]).unstack("config2")
        display(p_values)

        # table += "\\begin{tabular}{l" + "r"*len(candidates) + "} \\toprule \n"
        table += f"& \\multicolumn{{{len(candidates)}}}{{c}}{{{formatt(scenario)}}} \\\\ \\midrule \n"
        table += " & " + " & ".join([f"\\bf{{{formatt(h[1])}}}" for h in p_values.columns]) + "\\\\\n"
        table += "\\midrule \n"
        for config1, row in p_values.iterrows():
            significant = ["S" if p < 0.05 else "." for p in row]
            table += f"\\bf{{{formatt(str(config1))}}}" + " & " + " & ".join([f"{v:.2e} ({r})" if not np.isnan(v) else "" for v, r in zip(row, significant)]) + "\\\\ \n"
        table += "\\midrule \n"

    table += "\\bottomrule \n\\end{tabular}\n"

    # print(table)

    table_fn = Path(f"tables/mannwhitneyu_{quality}.tex")
    table_fn.parent.mkdir(exist_ok=True, parents=True)
    with open(table_fn, "w") as fh:
        fh.write(table)


In [ ]:
candidates = list(qdf["configurator"].unique())

partition = "test"
scenario = "MIP_CPLEX_REGIONS200_cutoff"

for quality in ["hypervolume", "igdp"]:
    table = "\\begin{tabular}{l" + "r"*len(candidates) + "} \\toprule \n"
    perf = qdf[(qdf["partition"] == partition)].set_index(["configurator", "scenario", "seed"])[["hypervolume","igdp"]].unstack("configurator")
    p_values = []
    for c1, c2 in itertools.product(list(qdf["configurator"].unique()), repeat=2):
        test = scipy.stats.mannwhitneyu(perf[(quality, c1)], perf[(quality, c2)], alternative="two-sided", method="exact")
        p_values.append({"config1": c1, "config2": c2, "p_value": test.pvalue})

    p_values = pd.DataFrame(p_values).set_index(["config1", "config2"]).unstack("config2")
    p_values

    # table += "\\begin{tabular}{l" + "r"*len(candidates) + "} \\toprule \n"
    table += f"& \\multicolumn{{{len(candidates)}}}{{c}}{{{formatt(quality)}}} \\\\ \\midrule \n"
    table += " & " + " & ".join([f"\\bf{{{formatt(h[1])}}}" for h in p_values.columns]) + "\\\\\n"
    table += "\\midrule \n"
    for config1, row in p_values.iterrows():
        significant = ["S" if p < 0.05 else "." for p in row]
        table += f"\\bf{{{formatt(str(config1))}}}" + " & " + " & ".join([f"{v:.2e} ({r})" if not np.isnan(v) else "" for v, r in zip(row, significant)]) + "\\\\ \n"

    table += "\\bottomrule \n\\end{tabular}\n"

    print(table)

    table_fn = Path(f"tables/mannwhitneyu_{quality}_combined.tex")
    table_fn.parent.mkdir(exist_ok=True, parents=True)
    with open(table_fn, "w") as fh:
        fh.write(table)

### Boxplots

In [ ]:
palette


In [ ]:
figsize = (4, 2.5)
xtickrotation = 0  # 35
# palette = {conf: col for conf, col in zip(configurators, color_palette)}
order = [conf for conf in list(config_names.values()) if conf in list(qdf["configurator"].unique())]
# for performance_measure, partition, (scenario, grp) in itertools.product(["hypervolume", "igdp"], ["validate", "test"], qdf.groupby("scenario")):
#     fig, ax = plt.subplots(1,1,figsize=figsize)
#     sns.boxplot(
#         data=grp[grp["partition"] == partition],
#         x=performance_measure,
#         y="configurator",
#         palette=palette,
#         order=order,
#     )
#     plt.xticks(rotation=xtickrotation)
#     #place legend outside top right corner of plot
#     # plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
#     # plt.title(f"{performance_measure:} variance on {partition} set {scenario}")
#     plt.tight_layout()
#     fn = Path(figurepath / f"boxplots/{performance_measure}_{partition}_{scenario}.pdf")
#     fn.parent.mkdir(exist_ok=True, parents=True)
#     plt.savefig(fn)
#     plt.close()
dpi = 300

def make_boxplot_figure(nqdf):
    figsize = (7, 2.5)
    
    fig = plt.figure(figsize=figsize, dpi=dpi)
    axes = fig.subplots(nrows=1, ncols=2, sharey=True, sharex=False)
    for i, (ax, performance_measure) in enumerate(zip(axes, ["hypervolume", "igdp"])):
        ax = sns.boxplot(
            data=nqdf[nqdf["partition"] == partition],
            x=performance_measure,
            y="configurator",
            palette=palette,
            order=order,
            ax=ax
        )

        #place legend outside top right corner of plot
        # plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        # plt.title(f"{performance_measure:} variance on all {partition} sets")
        # if performance_measure == "igdp":
        #     ax.set_xlim((0, 0.5))
        if i != 0:
            ax.set_ylabel(None)
        else:
            pass
            # labels = [config_names.get(item.get_text(), item.get_text()) for item in ax.get_yticklabels()]
            # print(labels)
            # ax.set_yticklabels(labels)
        ax.set_xlabel(performance_measure)
    fig.tight_layout()
    return fig

for partition in ["validate", "test"]:
    fig = make_boxplot_figure(nqdf)
    fn = Path(figurepath / f"boxplots/combined_{partition}.pdf")
    fn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(fn, bbox_inches="tight", dpi=dpi)
    plt.close()

for partition, (scenario, grp) in itertools.product(["validate", "test"], qdf.groupby("scenario")):
    fig = make_boxplot_figure(grp)
    fig.suptitle(scenario)
    fig.tight_layout()
    fn = Path(figurepath / f"boxplots/combined_{partition}_{scenario}.pdf")
    fn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(fn, bbox_inches="tight", dpi=dpi)
    plt.close()

In [ ]:
nqdf["configurator"].unique()

## ML F1-score ranking

In [ ]:
import robustranking as rr

In [ ]:
for scenario, partition in itertools.product(scenarios, partitions):
    scenario_name = str(scenario['experiment']["scenarios"])
    if scenario_name not in ["ML_RF_STUDENTS_precision_recall", "ML_RF_STUDENTS_precision_recall_size"]:
        continue

    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]

    # igdplus = igdp_dict[(scenario_name, partition)]

    df = scenario["run_result"]
    df = df[df["action"] == partition]

    df["f1"] = 2 * ((1 - df["precision"]) * (1 - df["recall"])) / ((1 - df["precision"]) + (1 - df["recall"]))


    print(scenario_name, partition)
    mdf = df.groupby(["configurators", "seeds"])["f1"].max().reset_index()
    print(mdf.groupby(["configurators"]).apply(lambda x: x["f1"].to_list()).to_dict())
    # display(df.groupby(["configurators"])["f1"].describe())#.sort_values(ascending=False))
    benchmark = rr.Benchmark().from_pandas(df.groupby(["configurators", "seeds"])["f1"].max().reset_index(), "configurators", "seeds", "f1")
    comp = rr.comparison.RankedComparison(benchmark, minimise=False)
    rr.utils.plots.plot_critical_difference(comp, alpha=0.5)

In [ ]:
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.multitest import multipletests

# Specify the algorithm to compare against
baseline_algorithm = 'SMAC'
alpha = 0.05
for scenario, partition in itertools.product(scenarios, partitions):
    scenario_name = str(scenario['experiment']["scenarios"])
    if scenario_name not in ["ML_RF_STUDENTS_precision_recall", "ML_RF_STUDENTS_precision_recall_size"]:
        continue

    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]

    # igdplus = igdp_dict[(scenario_name, partition)]

    df = scenario["run_result"]
    df = df[df["action"] == partition]

    df["f1"] = 2 * ((1 - df["precision"]) * (1 - df["recall"])) / ((1 - df["precision"]) + (1 - df["recall"]))

    print(scenario_name, partition)
    mdf = df.groupby(["configurators", "seeds"])["f1"].max().reset_index()
    observations = mdf.groupby(["configurators"]).apply(lambda x: x["f1"].to_list()).to_dict()

    # Perform statistical tests
    p_values_ttest = []
    p_values_wilcoxon = []
    algorithms = []
    
    for algorithm, data in observations.items():
        if algorithm != baseline_algorithm:
            # Paired t-test (one-sided)
            t_stat, p_value_ttest = ttest_rel(observations[baseline_algorithm], data, alternative='less')
            p_values_ttest.append(p_value_ttest)
            
            # Wilcoxon signed-rank test (one-sided)
            try:
                w_stat, p_value_wilcoxon = wilcoxon(observations[baseline_algorithm], data, alternative='less')
            except ValueError:
                w_stat, p_value_wilcoxon = np.nan, np.nan  # Handle edge cases (e.g., all zeros)
            p_values_wilcoxon.append(p_value_wilcoxon)
            
            # Store algorithms for reference
            algorithms.append(algorithm)
    
    # Apply multiple testing correction (Benjamini-Hochberg FDR)
    corrected_ttest = multipletests(p_values_ttest, method='holm')
    corrected_wilcoxon = multipletests(p_values_wilcoxon, method='holm',)
    
    # Store corrected results
    alpha = 0.05
    for i, algorithm in enumerate(algorithms):
        # results[algorithm] = {
        #     'ttestp': p_values_ttest[i],
        #     'ttestpc': corrected_ttest[1][i],
        #     'wilcoxonp': p_values_wilcoxon[i],
        #     'wilcoxonc': corrected_wilcoxon[1][i]
        # }

        print(f"{algorithm} VS. {baseline_algorithm}")
        print("\tt-test")
        print(f" > p-value (corrected): {corrected_ttest[1][i]} -> {'SIGNIFICANT' if corrected_ttest[1][i] < alpha else ''}")
        print("\tWilcoxon")
        print(f" > p-value (corrected): {corrected_ttest[1][i]} -> {'SIGNIFICANT' if corrected_ttest[1][i] < alpha else ''}")

In [ ]:
"df.groupby(["configurators", "seeds"])["f1"].max().reset_index().groupby("configurators")["f1"].describe()

benchmark = rr.Benchmark().from_pandas(df.groupby(["configurators", "seeds"])["f1"].max().reset_index(), "configurators", "seeds", "f1")
comp = rr.comparison.RankedComparison(benchmark, minimise=False)
rr.utils.plots.plot_critical_difference(comp)

In [ ]:
# ML precision recall with f1 score

# Find scenarios
for s in scenarios:
    scenario = s
    if s["experiment"]["scenarios"] == "ML_SVM_MNIST_precision_recall":
        break

df = scenario["run_result"]
df[df["configurators"] == "SMAC"]["configuration"]

df

# Performance with different run sizes

### Data Generation

In [ ]:
config_names

In [ ]:
n_samples = 190
list_n_runs = list(range(1,21))
hvs = []
igdp_dict = {}
partitions = ["validate", "test"]
for scenario, partition in itertools.product(scenarios, partitions):
    scenario_name = str(scenario['experiment']["scenarios"])
    print(scenario_name, partition)
    
    resultdir.joinpath("sampledruns").mkdir(exist_ok=True, parents=True)
    resultfile = resultdir / f"sampledruns/{scenario_name}_{partition}.csv"
    if resultfile.exists():
        print("LOAD CSV")
        ret = pd.read_csv(resultfile)
        hvs.append(ret)
        continue
    
    
    # if scenario_name not in ["MIP_CPLEX_REGIONS200_cutoff_50", "MIP_CPLEX_REGIONS200_runtime_50", "SAT_CMS_QUEENS_runtime_memory_50"]:
    #     continue
    print(scenario_name)
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]
    print(objectives)

    df = scenario["run_result"]
    # print(len(df[df["configurators"] == "default"]))
    # df = df[~((df["configurators"] == "default") & (df["seeds"] != 0))]
    # print(len(df[df["configurators"] == "default"]))
    if "ML_RF_STUDENTS_accuracy" in scenario_name:
        # Remove F1 from this scenario
        df = df[~((df["configurators"] == "SMAC") & (df["configuration"] == 2))]
    performances = df.groupby(["action","configurators", "seeds", "configuration"])[objectives].mean().unstack("action").swaplevel(0, 1, 1)

    minimum_val = performances["validate"].min()
    maximum_val = performances["validate"].max()

    minimum = performances[partition].min()
    maximum = performances[partition].max()

    print(f"{maximum=}")
    print(f"{minimum=}")

    points = performances[partition].to_numpy()
    #Normalize objectives
    for i, o in enumerate(objectives):
        points[:, i] = 1 + ((points[:, i] - minimum[o]) / (maximum[o] - minimum[o]))
    reference_set_ids = pygmo.fast_non_dominated_sorting(points)[0][0]
    reference_set = points[reference_set_ids, :]

    print(f"{reference_set.shape=}")

    igdplus = IGDPlus(reference_set)
    igdp_dict[(scenario_name, partition)] = igdplus

    # Sort configurators
    # SAMPLE DF HERE
    performances["order"] = performances.apply(lambda r: list(config_names.values()).index(r.name[0]) , axis=1)
    performances = performances.sort_values("order")

    # for runsize in range(2,20):
    def compute_performance(runsize: int) -> pd.DataFrame:
        all_runs = list(performances.reset_index()["seeds"].unique())
        all_combinations = list(itertools.combinations(all_runs, r=runsize))
        rng = np.random.RandomState(0)
        sample = rng.choice(len(all_combinations), size=min(n_samples, len(all_combinations)), replace=False)
        combinations = [all_combinations[i] for i in sample]

        results = []
        for ((_, configurator), pdf), sample_id in itertools.product(performances.groupby(["order", "configurators"]), range(len(combinations))):
            pdf = pdf.reset_index()
            # print(f"{sample_id=}, {combinations[sample_id]=}, {len(pdf)}")
            pdf = pdf[pdf["seeds"].isin(combinations[sample_id])]
            pdf = pdf.set_index(["seeds", "configuration"])
            # print(f"{len(pdf)}")

            # print(configurator)
            #Compute ND set from validation performance
            points = pdf["validate"].to_numpy()
            if len(points) > 1:
                ndf, dl, dc, ndr = pygmo.fast_non_dominated_sorting(points)
                front = ndf[0]
            else:
                front = [0]

            #Get test performance
            points = pdf[partition].to_numpy()
            if np.any(np.isnan(points)):
                print(f"MISSING POINTS, SKIPPING {configurator}")
                continue

            #Compute HV and IGD+
            #Normalize
            ndpoints = points[front, :]
            for i, o in enumerate(objectives):
                ndpoints[:, i] = 1 + ((ndpoints[:, i] - minimum[o]) / (maximum[o] - minimum[o]))

            hv = pygmo.hypervolume(ndpoints).compute([2.0+1e-1]*len(objectives))
            igdp_score = igdplus(ndpoints)
            res = {"scenario": scenario_name,
                        "configurator":configurator,
                        "partition": partition,
                        "hypervolume": hv,
                        "igdp": igdp_score,
                        "runsize": runsize,
                        "sample": sample_id}
            results.append(res)
        results = pd.DataFrame(results)
        return results
    
    with Pool() as pool:
        ret = pool.map(compute_performance, list_n_runs)
    pd.concat(ret).to_csv(resultfile, index=False)
    hvs.extend(ret)

qdf = pd.concat(hvs)
qdf.to_csv(resultdir / "sampledruns.csv", index=False)

In [ ]:
qdf = pd.concat([pd.read_csv(f) for f in resultdir.joinpath("sampledruns").iterdir()])
# qdf = pd.read_csv(resultdir / "sampledruns.csv")
qdf = rename_metadata(qdf)
qdf["configurator"] = qdf["configurator"].replace("Random Search 25", "Random Search (25)")
qdf["configurator"].unique()

### Plotting

In [ ]:
palette

In [ ]:
# markers = {config_names[k]: v for k, v in markers.items()}
# markers

In [ ]:
from matplotlib.ticker import MaxNLocator

qdf = pd.concat([pd.read_csv(f) for f in resultdir.joinpath("sampledruns").iterdir()])
qdf = rename_metadata(qdf)
qdf["configurator"] = qdf["configurator"].replace("Random Search 25", "Random Search").replace("Random Search (25)", "Random Search")
# qdf = qdf[~qdf["configurator"].isin(["Random Search"])]

sns.set(font_scale=1.)
sns.set_style("whitegrid")

# figsize = (3.5, 2.35)
dpi = 300

xticks = range(1, 23, 4)

for (scenario, grp), indicator in itertools.product(qdf.groupby("scenario"), ["hypervolume", "igdp"]):
    print(scenario, indicator)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(111)
    ax = sns.lineplot(
        data=grp[(grp["partition"] == "test")], 
        x="runsize", 
        y=indicator, 
        hue="configurator", 
        style="configurator", 
        ax=ax, 
        palette=palette, 
        linewidth=1.5, 
        dashes=False,
        markers=markers, 
        markersize=5,
        markeredgewidth=0,
    )
    # ax.set_title(scenario)
    ylabel = indicator
    if indicator == "igdp":
        ylabel = "IGD+"
    elif indicator == "hypervolume":
        ylabel = "Hypervolume"
    
    ax.set_title(f"{scenario.replace(' (c)','')}") #{ylabel.upper()}
    ax.get_legend().remove()
    ax.set_xlabel("n independent runs")
    ax.set_ylabel(ylabel)
    ax.set_xlim(1, 20)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    # ax.set_xticks(xticks)
    fig.tight_layout()
    figfn = Path(figurepath / f"vary_runs/{indicator}_{scenario}.pdf")
    figfn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(figfn, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.clf()
    plt.close()


#### Latex Inclusion

In [ ]:
# filenames = Path(figurepath / f"vary_runs").glob("*.pdf")
# 
# for filename in filenames:
# 
#     metric, scenario = filename.stem.split("_")
# 
#     if "hypervolume" in str(filename):
# 
#         # {indicator}_{scenario}
# 
#         filename2 = str(filename).replace("hypervolume", "igdp")
# 
#         replacements = dict(
#             floatpos = "h",
#             figwidth = "0.4",
#             figlabel = f"fig:vary_runs_{scenario}",
#             # figlabel2 = f"fig:vary_runs_{Path(filename2).stem}",
#             filename1 = str(filename),
#             filename2 = filename2, 
#             figcaption = f"Hypervolume (left) and IGD+ (right) for Scenario {scenario}",
#         )
# 
#         
# 
#         template = r"""
# \begin{figure}[floatpos]
#     \centering
#     \includegraphics[width=\textwidth]{figures/legend.pdf}\\
#     \vspace{-0.4cm}
#     \includegraphics[width=figwidth\textwidth]{filename1}\qquad
#     \includegraphics[width=figwidth\textwidth]{filename2}  
# \caption{figcaption}
# \label{figlabel}       
# \end{figure}
#         """
#         for k, v in replacements.items():
#             if "caption" in k:
#                 v = v.replace("_", "\_")
#             template = template.replace(k, v)
# 
#         print(template)
# 
# 
# 
# 
# 
#     """            \begin{subfigure}[t]{0.48\textwidth}
#                 \includegraphics[width=figwidth\textwidth]{filename1}\\
#                 \caption{figcaption1}
#                 \label{figlabel1}            
#             \end{subfigure}\hfill
#             \begin{subfigure}[t]{0.48\textwidth}
#                 \includegraphics[width=figwidth\textwidth]{filename2}\\
#                 \caption{figcaption2}
#                 \label{figlabel2}            
#             \end{subfigure}     
#     """


## Tables

In [ ]:
qdf
qdf["configurator"].unique()

In [ ]:
import scipy
from statsmodels.stats.multitest import multipletests

def formatt(s):
    return s.replace("_", "\_")

configorder = ["MO-SMAC-PHVI", "MO-SMAC-PE", "MO-ParamILS", "Random Search", "SMAC", "Default"]

fqdf = qdf[(qdf["partition"] == "test") & (qdf["runsize"] == 1)]


table = "\\begin{tabular}{l" + "r"*len(configorder) + "} \\toprule \n"
table += " & " + " & ".join([f"\\bf{{{formatt(c)}}}" for c in configorder]) + "\\\\\n"
table += "\\midrule \n"
for indicator, maximise in [("hypervolume", True), ("igdp", False)]:
    table += f"& \\multicolumn{{{len(configorder)}}}{{c}}{{{indicator}}} \\\\ \\midrule \n"
    all_rankings = []
    for scenario, grp in fqdf.groupby("scenario"):
        print(scenario, indicator)
        grp = grp.set_index(["configurator", "sample"])[indicator].unstack("configurator")[configorder]
        mean = grp.mean()
        # display(mean)
        best = np.argmax(mean) if maximise else np.argmin(mean)
        best_conf = mean.keys()[best]
        # display(best_conf)

        direction = -1 if maximise else 1
        ranking = 1 + np.argsort(np.argsort(direction*mean))
        all_rankings.append(ranking)
        # display(ranking)

        # statistical test
        values = grp.to_numpy().T
        # print(f"{values.shape=}")
        a = values[best]
        p_values = []
        for i in range(len(values)):
            b = values[i]

            # print(f"{a.shape=}, {b.shape=}, ")
            test = scipy.stats.mannwhitneyu(a, b, alternative="two-sided")
            p_values.append(test.pvalue)
        # display(p_values)
        del p_values[best] # remove self comparison
        # multiple test correction
        significant, _, _, _ = multipletests(p_values, alpha=0.05, method='holm', is_sorted=False, returnsorted=False)
        # display(significant)
        significant = list(significant)
        significant.insert(best, False)
        display(significant)
        #Now we can add the table line

        elements = []
        for i, (m, r, s) in enumerate(zip(mean, ranking, significant)):
            string = f"{m:.3f} ({r})"
            if i == best:
                string = f"\\textbf{{{string}}}"
            elif not s: # we cannot say it is significantly worse
                string = f"\\textbf{{{string}}}"
                # string += " \\dag"
            elements.append(string)

        table+= f"\\textbf{{{scenario}}} &" + " & ".join(elements) + " \\\\ \n"

    #Average rankings
    all_rankings = np.array(all_rankings)
    print(all_rankings)
    table += "\\multicolumn{1}{r}{\\it Average ranking} &" + " & ".join([f"{ar:.1f}" for ar in np.mean(all_rankings, axis=0)]) + "\\\\ \\midrule \n"

table += "\\bottomrule \n\\end{tabular}\n"

table_fn = Path(f"tables/resulttable_8runs.tex")
table_fn.parent.mkdir(exist_ok=True, parents=True)
with open(table_fn, "w") as fh:
    fh.write(table)

print(table)
#
# table = "\\begin{tabular}{l" + "r"*grp.shape[1] + "} \\toprule \n"
# table += " & " + " & ".join([f"\\bf{{{formatt(c)}}}" for c in configorder]) + "\\\\\n"
# table += "\\midrule \n"
# for indicator in ["hypervolume", "igdp"]:
#     continue
#
# print(table)


In [ ]:
# Generate Tables
def formatt(s):
    return s.replace("_", "\_")

fqdf = qdf[qdf["configurator"] != "MO-SMAC"]

for (performance_measure, maximise, notation), (p, grp) in itertools.product([("hypervolume", True, ".3f"), ("igdp", False, ".2e")], fqdf.groupby("partition")):
    print(p, performance_measure)
    grp = grp.set_index(["scenario", "configurator"])[performance_measure].unstack("configurator")
    grp = grp[configorder]
    display(grp)

    table = "\\begin{tabular}{l" + "r"*grp.shape[1] + "} \\toprule \n"
    table += " & " + " & ".join([f"\\bf{{{formatt(config_names[h])}}}" for h in grp.columns]) + "\\\\\n"
    table += "\\midrule \n"
    for scenario, row in grp.iterrows():
        best_val = row.max() if maximise else row.min()
        ranks = -row if maximise else row
        ranks = np.argsort(np.argsort(ranks))
        ranks = 1 + ranks
        table += f"\\bf{{{formatt(str(scenario))}}}" + " & " + " & ".join([f"{v:.2e} ({r})" if v != best_val or np.isnan(v) else f"\\bf{{{v:.2e}}} ({r})" for v, r in zip(row, ranks)]) + "\\\\\n"
    table += "\\bottomrule \n\\end{tabular}\n"

    table_fn = Path(f"tables/{performance_measure}_{p}.tex")
    table_fn.parent.mkdir(exist_ok=True, parents=True)
    with open(table_fn, "w") as fh:
        fh.write(table)

# Configurator run statistics

In [ ]:
resultdir = Path("results")

df = []
for d in tqdm(resultdir.joinpath("configure").iterdir()):
    # print(f"> {d}")
    cont = False
    for conf in ["MO-SMAC", "ParEGO", "SMAC", "MO-SMAC-PHVI"]:
    # for conf in ["SAT-CMS-QUEENS-runtime-memory_SMAC_"]:
        if conf in str(d):
            #print(f"\t{conf} in there")
            cont = True
    if not cont:
        continue

    if "test" in str(d):
        continue

    try:
        with open(d, "rb") as fh:
            r = pickle.load(fh)
            # print(r["experiment"])
        if r["experiment"]["configurators"] == "SMAC":
            # print(r["run_result"])
            # input()
            continue
        runhistory = r["run_result"]["runhistory"]
        runningtime = 0
        diff = []
        for trialkey, trial in runhistory.items():
            runningtime += trial.endtime - trial.starttime
            diff += [trial.starttime, trial.endtime]
        #
        # print(f"ta runtime: {runningtime / 3600} hours")
        # print(f"{len(runhistory)=}")
        # print(f"Configurations: {runhistory._n_id}")

        diff = np.array(diff)
        overhead = np.diff(diff)[1::2]

        r["experiment"]["taruntime"] = runningtime
        r["experiment"]["ntrials"] = len(runhistory)
        r["experiment"]["nconfigs"] = runhistory._n_id
        r["experiment"]["overhead"] = sum(np.sort(overhead)[::-1])
        r["experiment"]["configgens"] = np.count_nonzero(overhead > 0.25)

        origins = [runhistory.get_config(i).origin for i in range(1, runhistory._n_id+1)]
        for org, count in zip(*np.unique(origins, return_counts=True)):
            r["experiment"][org] = count / runhistory._n_id

        df.append(r["experiment"])
    except ModuleNotFoundError:
        pass
        # print(f"{d} uses old version of SMAC")

df = pd.DataFrame(df)

In [ ]:
df

## Table

In [ ]:
#Statistics
df["refreshfreq"] = df["nconfigs"] / df["configgens"]
tdf = df.groupby(["scenarios", "configurators"]).agg(["mean"]).reset_index()
# sns.boxplot(date=tdf, x=[""])
tdf
# No.times Retraining of acquisition model

In [ ]:
cols = ["scenarios", "configurators", "taruntime", "ntrials", "nconfigs", "Acquisition Function Maximizer: Local Search", "Acquisition Function Maximizer: Random Search (sorted)", "Random Search"]

def formatt(s):
    return s.replace("_", "\_")

table = "\\begin{tabular}{l" + "r"*len(cols) + "} \\toprule \n"
table += " & " + " & ".join([f"\\bf{{{formatt(h)}}}" for h in cols]) + "\\\\\n"
table += "\\midrule \n"
for _, grp in tdf[cols].iterrows():
    values = []
    for key, val in grp.to_dict().items():
        if isinstance(val, str):
            values.append(formatt(val))
        else:
            values.append(f"{val:.2f}")

    table += " & ".join(values) + "\\\\\n"
table += "\\bottomrule \n\\end{tabular}\n"

with open(f"tables/configurator_stats.tex", "w") as fh:
        fh.write(table)

print(table)

## Run analyses

In [ ]:
t = r["run_result"]["trajectory"]
trajectory = t["trajectory"]
trajectory_index = 0
incumbent = trajectory[trajectory_index]

unique_isb_pairs = set()
incumbent_run = 0
config_run = 0
progress = []

config_trial_counter = {}

for trial_id, (tinfo, tresult) in enumerate(runhistory.items()):
    challenger_added_to_incumbent = False

    if trial_id == 0:
        print(trial_id)
        print(tinfo)
        print(tresult)
        continue

    if tinfo.config_id in incumbent["config_ids"]:
        incumbent_run += 1
    else:
        config_run += 1
        if tinfo.config_id not in config_trial_counter:
            config_trial_counter[tinfo.config_id] = 0
        config_trial_counter[tinfo.config_id] += 1

    isb_key = f"{tinfo.instance}_{tinfo.seed}"
    unique_isb_pairs.add(isb_key)

    #Incumbent changes after running a trial
    if trial_id >= incumbent["trial"]:
        trajectory_index = min(trajectory_index+1, len(trajectory)-1)
        incumbent = trajectory[trajectory_index]
        if tinfo.config_id in incumbent["config_ids"]:
            challenger_added_to_incumbent = True

    stats = {
        "unique_isb_keys": len(unique_isb_pairs),
        "incumbent_run": incumbent_run,
        "config_run": config_run,
        "incumbent_size": len(incumbent["config_ids"]),
        "challenger_added_to_incumbent": challenger_added_to_incumbent
    }

    progress.append(stats)

progress = pd.DataFrame(progress)

progress["incumbent_size"].plot()

np.unique(list(config_trial_counter.values()), return_counts=True)

#progress[progress["challenger_added_to_incumbent"]]

In [ ]:
def analyse_runhistory_isb_keys(r):
    print("Experiment:", r["experiment"]["scenarios"], r["experiment"]["configurators"],  r["experiment"]["seeds"], )
    print(r["run_result"].keys())
    t = r["run_result"]["trajectory"]
    trajectory = t["trajectory"]
    runhistory = r["run_result"]["runhistory"]

    config_trial_counter_seq = []
    config_trial_counter_all = {}
    current_config = np.nan
    for trial_id, (tinfo, tresult) in enumerate(runhistory.items()):
        if tinfo.config_id is not current_config:
            config_trial_counter_seq.append(0)
            current_config = tinfo.config_id

        config_trial_counter_seq[-1] += 1

        if tinfo.config_id not in config_trial_counter_all:
            config_trial_counter_all[tinfo.config_id] = set()
        config_trial_counter_all[tinfo.config_id].add(f"{tinfo.instance}_{tinfo.seed}")

    print("Run status counts")
    rlist = [tresult.status for _, tresult in runhistory.items()]
    print(list(zip(*np.unique(rlist, return_counts=True))))
    print("Sequential trial count of a config")
    print(list(zip(*np.unique(config_trial_counter_seq, return_counts=True))))
    print("All trials counts per config")
    hist_count = list(zip(*np.unique([len(c) for c in config_trial_counter_all.values()], return_counts=True)))
    print(hist_count)
    # for n_trials, count in hist_count:
    #     print(f"{n_trials:3}: {count}")
    print("Unique ISB keys")
    all_unique_keys = set()
    for isb_keys in config_trial_counter_all.values():
        all_unique_keys = all_unique_keys.union(isb_keys)
    print(len(all_unique_keys))

    maximum_runs = max([h[0] for h in hist_count])
    intersection_keys = all_unique_keys
    print(f"Intersection between config with most runs ({maximum_runs})")
    for isb_keys in config_trial_counter_all.values():
        if len(isb_keys) == maximum_runs:
            intersection_keys = intersection_keys.intersection(isb_keys)
    print(len(intersection_keys))

    for inc_id, incumbent in enumerate(trajectory):
        incumbent_trials = {i:set() for i in incumbent["config_ids"]}

        for trial_id, (tinfo, tresult) in enumerate(runhistory.items()):
            if trial_id == incumbent["trial"]:
                break

            if tinfo.config_id in incumbent["config_ids"]:
                incumbent_trials[tinfo.config_id].add(f"{tinfo.instance}_{tinfo.seed}")

        union_isb = set()
        intersect_isb = all_unique_keys

        for isb_keys in incumbent_trials.values():
            union_isb = union_isb.union(isb_keys)
            intersect_isb = intersect_isb.intersection(isb_keys)

        print(f"Incumbent {inc_id} of size {len(incumbent_trials)} ran on {len(union_isb)} different isb_keys which share {len(intersect_isb)} isb_keys")
        for config_id, isb_keys in incumbent_trials.items():
            overlap_with_others = [len(isb_keys.intersection(k)) for cid, k in incumbent_trials.items() if cid is not config_id]
            if len(overlap_with_others) == 0:
                overlap_with_others = [0]

            print(f"\t Config {config_id:4} has {len(isb_keys):3} runs. Maximum overlap pair {max(overlap_with_others)}")
analyse_runhistory_isb_keys(r)

In [ ]:
import contextlib
scp = [(s,c) for s,c in itertools.product(df["scenarios"].unique().tolist(), df["configurators"].unique().tolist())]

scp = set(scp)

for d in tqdm(resultdir.joinpath("configure").iterdir()):
    cont = False
    for c in ["SMAC", "ParEGO"]:
        if c in str(d):
            cont = True

    if not cont:
        continue

    try:
        with open(d, "rb") as fh:
            r = pickle.load(fh)

            sck = (r["experiment"]["scenarios"], r["experiment"]["configurators"])
            if sck in scp:
                scp.remove(sck)

                with open(f"trajectory_analysis/{sck[0]}_{sck[1]}_analysis.txt","w") as wfh:
                    with contextlib.redirect_stdout(wfh):
                        analyse_runhistory_isb_keys(r)
    except:
        pass

